In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('AUG20-21data.txt')
winter2 = pd.read_csv ('AUG21-22data.txt')
winter3 = pd.read_csv ('AUG22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
aug = LocationInfo(
    'Augusta Lake', 'Maine', 
    latitude= 44.3165,
    longitude= -69.7681,
    timezone='America/Connecticut'
)

aug

LocationInfo(name='Augusta Lake', region='Maine', timezone='America/Connecticut', latitude=44.3165, longitude=-69.7681)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(aug.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(aug.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(aug.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:38  2020-12-01 16:00:48  2020-12-01
1   2020-12-02 06:56:44  2020-12-02 16:00:29  2020-12-02
2   2020-12-03 06:57:48  2020-12-03 16:00:13  2020-12-03
3   2020-12-04 06:58:52  2020-12-04 15:59:59  2020-12-04
4   2020-12-05 06:59:54  2020-12-05 15:59:48  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:24:27  2021-02-24 17:20:37  2021-02-24
86  2021-02-25 06:22:48  2021-02-25 17:21:58  2021-02-25
87  2021-02-26 06:21:07  2021-02-26 17:23:18  2021-02-26
88  2021-02-27 06:19:26  2021-02-27 17:24:38  2021-02-27
89  2021-02-28 06:17:45  2021-02-28 17:25:57  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:21  2021-12-01 16:00:52  2021-12-01
1   2021-12-02 06:56:28  2021-12-02 16:00:33  2021-12-02
2   2021-12-03 06:57:33  2021-12-03 16:00:16  2021-12-03
3   2021-12-04 06:58:37  2021-12-04 16:00:02  2021-12-04
4   2021-12-05 06:59:39  2021-12-05 15:59:50  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:24:51  2022-02-24 17:20:18  2022-02-24
86  2022-02-25 06:23:12  2022-02-25 17:21:38  2022-02-25
87  2022-02-26 06:21:32  2022-02-26 17:22:58  2022-02-26
88  2022-02-27 06:19:51  2022-02-27 17:24:18  2022-02-27
89  2022-02-28 06:18:09  2022-02-28 17:25:38  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:05  2022-12-01 16:00:57  2022-12-01
1   2022-12-02 06:56:12  2022-12-02 16:00:37  2022-12-02
2   2022-12-03 06:57:17  2022-12-03 16:00:20  2022-12-03
3   2022-12-04 06:58:21  2022-12-04 16:00:05  2022-12-04
4   2022-12-05 06:59:24  2022-12-05 15:59:52  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:25:14  2023-02-24 17:19:58  2023-02-24
86  2023-02-25 06:23:36  2023-02-25 17:21:19  2023-02-25
87  2023-02-26 06:21:56  2023-02-26 17:22:39  2023-02-26
88  2023-02-27 06:20:15  2023-02-27 17:23:59  2023-02-27
89  2023-02-28 06:18:34  2023-02-28 17:25:18  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AUG,2020-12-01 00:53:00,8.13,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
1,AUG,2020-12-01 01:53:00,7.11,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
2,AUG,2020-12-01 02:53:00,6.86,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
3,AUG,2020-12-01 03:53:00,5.84,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
4,AUG,2020-12-01 04:53:00,4.83,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
...,...,...,...,...,...,...,...
267,AUG,2021-02-27 18:53:00,1.27,UP BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
268,AUG,2021-02-27 19:53:00,2.54,UP BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
269,AUG,2021-02-27 20:53:00,1.02,RA BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
270,AUG,2021-02-27 21:53:00,2.29,-RA BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AUG,2021-12-02 11:53:00,0.0001,-SN,2021-12-02,2021-12-02 06:56:28,2021-12-02 16:00:33
1,AUG,2021-12-02 12:53:00,0.2500,UP BR,2021-12-02,2021-12-02 06:56:28,2021-12-02 16:00:33
2,AUG,2021-12-02 13:53:00,0.5100,-FZRA BR,2021-12-02,2021-12-02 06:56:28,2021-12-02 16:00:33
3,AUG,2021-12-02 14:53:00,0.7600,-FZRA BR,2021-12-02,2021-12-02 06:56:28,2021-12-02 16:00:33
4,AUG,2021-12-02 15:53:00,0.7600,-RA BR,2021-12-02,2021-12-02 06:56:28,2021-12-02 16:00:33
...,...,...,...,...,...,...,...
274,AUG,2022-02-25 22:53:00,0.0001,-SN,2022-02-25,2022-02-25 06:23:12,2022-02-25 17:21:38
275,AUG,2022-02-25 23:53:00,0.0001,-SN,2022-02-25,2022-02-25 06:23:12,2022-02-25 17:21:38
276,AUG,2022-02-26 01:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:21:32,2022-02-26 17:22:58
277,AUG,2022-02-27 22:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:19:51,2022-02-27 17:24:18


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AUG,2022-12-01 00:53:00,4.8300,-RA BR,2022-12-01,2022-12-01 06:55:05,2022-12-01 16:00:57
1,AUG,2022-12-01 01:53:00,9.1400,RA BR,2022-12-01,2022-12-01 06:55:05,2022-12-01 16:00:57
2,AUG,2022-12-01 02:53:00,1.7800,-RA,2022-12-01,2022-12-01 06:55:05,2022-12-01 16:00:57
3,AUG,2022-12-01 03:53:00,0.2500,-RA,2022-12-01,2022-12-01 06:55:05,2022-12-01 16:00:57
4,AUG,2022-12-01 18:53:00,0.0001,-SN,2022-12-01,2022-12-01 06:55:05,2022-12-01 16:00:57
...,...,...,...,...,...,...,...
266,AUG,2023-02-26 17:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:21:56,2023-02-26 17:22:39
267,AUG,2023-02-26 18:53:00,0.2500,-SN,2023-02-26,2023-02-26 06:21:56,2023-02-26 17:22:39
268,AUG,2023-02-26 19:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:21:56,2023-02-26 17:22:39
269,AUG,2023-02-26 20:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:21:56,2023-02-26 17:22:39


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,AUG,2020-12-01 08:53:00,0.0001,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
8,AUG,2020-12-01 09:53:00,1.7800,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
9,AUG,2020-12-01 10:53:00,0.5100,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
10,AUG,2020-12-01 12:53:00,5.0800,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
11,AUG,2020-12-01 15:53:00,2.2900,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
...,...,...,...,...,...,...,...
244,AUG,2021-02-16 12:53:00,2.5400,UP BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44
245,AUG,2021-02-16 13:53:00,2.7900,-SN BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44
246,AUG,2021-02-16 14:53:00,1.2700,-FZRA BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44
247,AUG,2021-02-16 15:53:00,2.5400,UP FZFG,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
7,AUG,2020-12-01 08:53:00,0.0001,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48,2020-12-01 07:55:38,2020-12-01 15:00:48
8,AUG,2020-12-01 09:53:00,1.7800,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48,2020-12-01 07:55:38,2020-12-01 15:00:48
9,AUG,2020-12-01 10:53:00,0.5100,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48,2020-12-01 07:55:38,2020-12-01 15:00:48
10,AUG,2020-12-01 12:53:00,5.0800,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48,2020-12-01 07:55:38,2020-12-01 15:00:48
17,AUG,2020-12-05 13:53:00,0.0001,-RA,2020-12-05,2020-12-05 06:59:54,2020-12-05 15:59:48,2020-12-05 07:59:54,2020-12-05 14:59:48
...,...,...,...,...,...,...,...,...,...
243,AUG,2021-02-16 11:53:00,1.5200,UP BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44,2021-02-16 07:37:06,2021-02-16 16:09:44
244,AUG,2021-02-16 12:53:00,2.5400,UP BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44,2021-02-16 07:37:06,2021-02-16 16:09:44
245,AUG,2021-02-16 13:53:00,2.7900,-SN BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44,2021-02-16 07:37:06,2021-02-16 16:09:44
246,AUG,2021-02-16 14:53:00,1.2700,-FZRA BR,2021-02-16,2021-02-16 06:37:06,2021-02-16 17:09:44,2021-02-16 07:37:06,2021-02-16 16:09:44


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.8294859154929578
2021-2022: 0.3342186813186813
2022-2023: 1.0953616438356164


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,AUG,2020-12-01 00:53:00,8.13,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
1,AUG,2020-12-01 01:53:00,7.11,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
2,AUG,2020-12-01 02:53:00,6.86,-RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
3,AUG,2020-12-01 03:53:00,5.84,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
4,AUG,2020-12-01 04:53:00,4.83,RA BR,2020-12-01,2020-12-01 06:55:38,2020-12-01 16:00:48
...,...,...,...,...,...,...,...
267,AUG,2021-02-27 18:53:00,1.27,UP BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
268,AUG,2021-02-27 19:53:00,2.54,UP BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
269,AUG,2021-02-27 20:53:00,1.02,RA BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38
270,AUG,2021-02-27 21:53:00,2.29,-RA BR,2021-02-27,2021-02-27 06:19:26,2021-02-27 17:24:38


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_14280\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.9282656249999999
2021-2022: 0.47922465753424665
2022-2023: 0.6101869565217392


In [32]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('aug.csv', index=False)